In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

import explore
import modeling

df = explore.make_initial_df()
df = explore.add_new_columns(df)

df = explore.bin_link_counts(df)
df = explore.bin_word_counts(df)

# make vectorized_df
vectorized_df = explore.make_vectorized_df(df)
# add new columns to vectorized_df
vectorized_df = explore.aggregate_columns(vectorized_df)
vectorized_df.head()

# scale and make test predictions dfs
X_train_scaled, X_test_scaled, y_train, y_test, train_predictions, test_predictions = explore.get_splits(df, vectorized_df)
# reduce the vectorized dataframe 
X_train_reduced, X_test_reduced = explore.prep_vectorized_df(X_train_scaled, X_test_scaled)

train_predictions.head()

# make predictions
train_predictions, test_predictions = modeling.make_predictions_df(X_train_reduced, X_test_reduced, y_train, train_predictions, test_predictions)

train_predictions.head()

evaluation_report = modeling.train_evaluation(train_predictions)

def test_evaluation(test_predictions):
    # Logistic regression accuracy score, confustion matrix, classification report for test data
    print('Evaluation Metrics for Logistic Regression Model')
    print()
    print()
    print('Accuracy: {:.2%}'.format(accuracy_score(test_predictions.actual, test_predictions.lr_predictions)))
    print('----------------------------------------------------------------------------------------------')
    print('Confusion Matrix')
    print(pd.crosstab(test_predictions.lr_predictions, test_predictions.actual))
    print('----------------------------------------------------------------------------------------------')
    print(classification_report(test_predictions.actual, test_predictions.lr_predictions))

    print()
    print()
    print()
    print()
    print()

test_evaluation(test_predictions)

df.link_bins

In [12]:
# make train and test precitions dataframe starting with actual values
# train_predictions = pd.DataFrame(y_train)
# train_predictions.columns = ['actual']
# test_predictions = pd.DataFrame(y_test)
# test_predictions.columns = ['actual']

# add baseline to predictions dataframe
# train_predictions['baseline'] = 'Python'
# test_predictions['baseline'] = 'Python'
# train_predictions

# test_predictions

# train, test = modeling.logistic_regression(df, vectorized_df)

# train

# rf = RandomForestClassifier(bootstrap=True, 
#                             class_weight=None, 
#                             criterion='gini',
#                             min_samples_leaf=3,
#                             n_estimators=100,
#                             max_depth=3, 
#                             random_state=123)

# rf.fit(X_train, y_train)

# train_random_forest_predictions = rf.predict(X_train)
# test_random_forest_predictions = rf.predict(X_test)

# knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

# knn.fit(X_train, y_train)

# train_knn_predictions = knn.predict(X_train)
# test_knn_predictions = knn.predict(X_test)

# X_train_scaled, X_test_scaled, X_train_reduced, X_test_reduced = explore.prep_vectorized_df(df, vectorized_df)

# X_train_scaled, X_test_scaled, X_train_reduced, X_test_reduced = modeling.make_predictions_df(df, vectorized_df)

